# Setup 


In [ ]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
import json
from dotenv import load_dotenv
from langdetect import detect

load_dotenv()

True

# NEED TO ADD CITATIONS!!!!!!!!!1

use https://python.langchain.com/docs/how_to/qa_citations/

In [3]:

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    messages=[
        {
            "role": "user",
            "content": """Hello!"""
        }
    ],
    temperature=0.6
)

print(completion.to_json())


{
  "id": "chatcmpl-5201472be1a3433cbcf14a68c96e3330",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning_content": null
      },
      "stop_reason": null
    }
  ],
  "created": 1748194898,
  "model": "meta-llama/Meta-Llama-3.1-70B-Instruct",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 12,
    "total_tokens": 22,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "prompt_logprobs": null
}


In [ ]:
llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0.6
)

multilingual_llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="Qwen/Qwen3-32B",
    temperature=0.6
)

In [8]:
# Create a simple prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("assistant", "How can I assist you today?"),
    ("human", "{question}")
])

# Create a simple chain
chain = prompt | llm

# Test it
response = chain.invoke({"question": "Hello!"})
print(response.content)

Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


# CP 1. Climate legislation Pipeline

##### Indicator CP 1.a Does the country have a framework climate law or equivalent?

Individualy test the following (A country is assessed as ‘Yes’ if it has a framework climate law that fulfils all of
the following):

1. It sets a strategic direction for decarbonisation (i.e. it must include a clear
statement to meet the goals of the Paris Agreement or a national long-term
decarbonisation target)

2. It is enshrined in law (i.e. it must be legislative rather than executive, except in
particular political systems)

3. It sets out at least one of the following obligations: meeting a national target;
developing, revising, implementing or complying with domestic plans,
strategies or policies; developing policy instruments such as regulation, taxation
or public spending in support of climate change goals

4. Also check this In exceptional cases, the combination of a broad environmental law and a clearly
linked executive climate strategy may be sufficient to meet these criteria

**Then have other model check if 1,2,3 are satisfied or 4 is satisfied -> answer yes**

**Then have another model format output into markdown using (this)[https://python.langchain.com/docs/tutorials/extraction/]**



### 1.1 Chain of thought Implementation
#### 1.1.1 Prompting

In [10]:
# Define individual criterion evaluation prompts
criterion_1_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a climate law sets a strategic direction for decarbonisation.
    
    A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
    OR a national long-term decarbonisation target.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this law set a strategic direction for decarbonisation?")
])

criterion_2_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a climate law is enshrined in law.
    
    A law meets this criterion if it is legislative rather than executive 
    (except in particular political systems where executive action has legal force).
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nIs this law enshrined in law (legislative rather than executive)?")
])

criterion_3_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a climate law sets out specific obligations.
    
    A law meets this criterion if it sets out at least one of the following:
    - Meeting a national target
    - Developing, revising, implementing or complying with domestic plans, strategies or policies
    - Developing policy instruments such as regulation, taxation or public spending in support of climate goals
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this law set out the required obligations?")
])

criterion_4_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate the exceptional case criterion.
    
    This criterion is met if there is a combination of:
    - A broad environmental law AND
    - A clearly linked executive climate strategy
    
    This combination may be sufficient to meet the framework criteria in exceptional cases.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this represent a valid exceptional case (broad environmental law + linked executive strategy)?")
])

# Final assessment prompt
final_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' for having framework climate law if:
    - Criteria 1, 2, AND 3 are all satisfied, OR
    - Criterion 4 is satisfied (exceptional case)
    
    Based on the individual assessments, provide a final 'YES' or 'NO' answer with reasoning."""),
    ("human", """Individual criterion assessments:
    Criterion 1 (Strategic direction): {criterion_1_result}
    Criterion 2 (Enshrined in law): {criterion_2_result}
    Criterion 3 (Obligations): {criterion_3_result}
    Criterion 4 (Exceptional case): {criterion_4_result}
    
    What is the final assessment?""")
])

#### 1.1.2 Retrieve Context


#### 1.1.4 Run

In [ ]:
# Standard chains
criterion_1_chain = criterion_1_prompt | llm
criterion_2_chain = criterion_2_prompt | llm
criterion_3_chain = criterion_3_prompt | llm
criterion_4_chain = criterion_4_prompt | llm

# Multilingual chains
criterion_1_chain_noneng = criterion_1_prompt | multilingual_llm
criterion_2_chain_noneng = criterion_2_prompt | multilingual_llm
criterion_3_chain_noneng = criterion_3_prompt | multilingual_llm
criterion_4_chain_noneng = criterion_4_prompt | multilingual_llm


# Create an enhanced evaluation function that routes based on language
def evaluate_all_criteria_multilingual(inputs):
    """Evaluate all four criteria using appropriate LLM based on language detection"""
    
    # Detect languages for each context
    lang_1 = detect_language(inputs["context_1"])
    lang_2 = detect_language(inputs["context_2"])
    lang_3 = detect_language(inputs["context_3"])
    lang_4 = detect_language(inputs["context_4"])
    
    # Helper function to choose appropriate chain
    def get_chain_for_language(language, standard_chain, multilingual_chain):
        return multilingual_chain if language != 'en' else standard_chain
    
    # Route to appropriate chains based on language
    chain_1 = get_chain_for_language(lang_1, criterion_1_chain, criterion_1_chain_noneng)
    chain_2 = get_chain_for_language(lang_2, criterion_2_chain, criterion_2_chain_noneng)
    chain_3 = get_chain_for_language(lang_3, criterion_3_chain, criterion_3_chain_noneng)
    chain_4 = get_chain_for_language(lang_4, criterion_4_chain, criterion_4_chain_noneng)
    
    # Execute evaluations
    criterion_1_result = chain_1.invoke({"context": inputs["context_1"]})
    criterion_2_result = chain_2.invoke({"context": inputs["context_2"]})
    criterion_3_result = chain_3.invoke({"context": inputs["context_3"]})
    criterion_4_result = chain_4.invoke({"context": inputs["context_4"]})
    
    return {
        "criterion_1_result": criterion_1_result.content,
        "criterion_2_result": criterion_2_result.content,
        "criterion_3_result": criterion_3_result.content,
        "criterion_4_result": criterion_4_result.content,
        "languages_detected": {
            "context_1": lang_1,
            "context_2": lang_2,
            "context_3": lang_3,
            "context_4": lang_4
        }
    }

# Update the complete assessment chain
complete_assessment_chain_multilingual = (
    RunnableLambda(evaluate_all_criteria_multilingual)
    | final_assessment_prompt 
    | llm  # Final assessment can stay in English
)

Final Assessment:
Based on the individual assessments, I conclude that the country has a framework climate law: **YES**

The reason is that all three main criteria (1, 2, and 3) are satisfied. The law provides a clear strategic direction, is enshrined in law, and imposes obligations on the country to develop national climate plans and carbon pricing mechanisms. Since Criterion 4 is not satisfied, it is not an exceptional case, and we rely on the satisfaction of the first three criteria.


In [ ]:
# Run the complete pipeline
result = complete_assessment_chain_multilingual.invoke(test_contexts)
#some function that takes indicator, retrieves context and then runs complete_assessment_chain fucntion
print("Final Assessment:")
print(result.content)

### 1.1.5 Add citations

Try generation post processing langchain approach

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

# Data models for your existing chunk metadata
class ChunkMetadata(BaseModel):
    """Metadata for a text chunk used in generation"""
    chunk_id: int = Field(..., description="Unique identifier for this chunk")
    document_name: str = Field(..., description="Name of the source document")
    page_number: int = Field(..., description="Page number in the document")
    paragraph_number: int = Field(..., description="Paragraph number on the page")
    chunk_text: str = Field(..., description="The actual text content of the chunk")

# Citation model for sentence-level citations
class SentenceCitation(BaseModel):
    """Citation information for a single sentence"""
    document_name: str = Field(..., description="Name of the source document")
    page_number: int = Field(..., description="Page number in the document") 
    paragraph_number: int = Field(..., description="Paragraph number on the page")
    
    def format_citation(self) -> str:
        """Format citation as inline text"""
        return f"({self.document_name}, p. {self.page_number}, ¶{self.paragraph_number})"

# Model for a sentence with its citation
class CitedSentence(BaseModel):
    """A sentence paired with its supporting citation"""
    sentence_text: str = Field(..., description="The complete sentence text")
    citation: SentenceCitation = Field(..., description="Citation supporting this sentence")
    
    def format_cited_sentence(self) -> str:
        """Format sentence with inline citation"""
        return f"{self.sentence_text} {self.citation.format_citation()}"

# Main structured output model
class AnnotatedText(BaseModel):
    """Complete text broken into sentences with citations"""
    cited_sentences: List[CitedSentence] = Field(
        ..., 
        description="List of sentences from the generated text, each with its supporting citation"
    )
    
    def format_full_text(self) -> str:
        """Reconstruct the full text with citations after each sentence"""
        return " ".join([cs.format_cited_sentence() for cs in self.cited_sentences])

In [ ]:

# Helper function to format chunks for LLM context
def format_chunks_with_metadata(chunks: List[ChunkMetadata]) -> str:
    """Format chunks with IDs and metadata for LLM processing"""
    formatted_chunks = []
    for chunk in chunks:
        chunk_info = f"""
Chunk ID: {chunk.chunk_id}
Document: {chunk.document_name}
Page: {chunk.page_number}
Paragraph: {chunk.paragraph_number}
Content: {chunk.chunk_text}
"""
        formatted_chunks.append(chunk_info)
    
    return "\n" + "="*50 + "\n".join(formatted_chunks)

# LangChain integration function
def create_citation_prompt(generated_text: str, chunks: List[ChunkMetadata]) -> str:
    """Create the prompt for citation annotation"""
    formatted_chunks = format_chunks_with_metadata(chunks)
    
    system_prompt = f"""You are tasked with adding citations to generated text. 

AVAILABLE SOURCE CHUNKS:
{formatted_chunks}

INSTRUCTIONS:
1. Break the generated text into individual sentences
2. For each sentence, identify which chunk(s) from the available sources best support that sentence
3. Create a citation using the document name, page number, and paragraph number
4. Return the structured output with each sentence paired with its citation

GENERATED TEXT TO ANNOTATE:
{generated_text}

Please return each sentence with its appropriate citation based on the source chunks provided."""
    
    return system_prompt

# Example usage structure for LangChain integration
def annotate_with_citations(generated_text: str, chunks: List[ChunkMetadata], llm):
    """
    Main function to add citations to generated text using LangChain
    
    Args:
        generated_text: The text generated by your LLM
        chunks: List of chunks with metadata that were used for generation
        llm: Your LangChain LLM instance with structured output
    """
    
    # Create structured LLM
    structured_llm = llm.with_structured_output(AnnotatedText)
    
    # Create prompt
    prompt_text = create_citation_prompt(generated_text, chunks)
    
    # Get structured response
    response = structured_llm.invoke([("human", prompt_text)])
    
    # Return formatted text with citations
    return response.format_full_text()

# Example of how your data might look
example_chunks = [
    ChunkMetadata(
        chunk_id=1,
        document_name="Climate Report 2024",
        page_number=15,
        paragraph_number=3,
        chunk_text="Global temperatures have risen by 1.2°C since pre-industrial times."
    ),
    ChunkMetadata(
        chunk_id=2,
        document_name="Climate Report 2024", 
        page_number=23,
        paragraph_number=1,
        chunk_text="Sea levels are projected to rise between 0.5 and 2.5 meters by 2100."
    )
]

example_generated_text = "Climate change has caused significant global warming in recent decades. Ocean levels are expected to increase dramatically over the next century."

#### 1.2 Alternative Large context window Implementation

In [ ]:
# Load larger context LLM
large_context_llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Llama-3.3-70B-Instruct",
    temperature=0.6
)

In [ ]:
# Set context to whole entire corpus of relevant documetns according to slyvan sheet

all_documents_context =

In [ ]:
# Single comprehensive climate legislation assessment prompt
comprehensive_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation assessment. 

Your task is to evaluate whether a country has a framework climate law based on specific criteria and provide a structured markdown assessment.

EVALUATION CRITERIA:
A country is assessed as 'YES' if it has a framework climate law that fulfils ALL of criteria 1, 2, AND 3, OR criterion 4:

1. STRATEGIC DIRECTION: Sets a strategic direction for decarbonisation (must include a clear statement to meet the goals of the Paris Agreement OR a national long-term decarbonisation target)

2. ENSHRINED IN LAW: Is enshrined in law (must be legislative rather than executive, except in particular political systems)

3. OBLIGATIONS: Sets out at least one of the following obligations:
   - Meeting a national target
   - Developing, revising, implementing or complying with domestic plans, strategies or policies
   - Developing policy instruments such as regulation, taxation or public spending in support of climate goals

4. EXCEPTIONAL CASE: The combination of a broad environmental law AND a clearly linked executive climate strategy may be sufficient to meet these criteria

ASSESSMENT LOGIC:
- If criteria 1, 2, AND 3 are all satisfied → YES
- If criterion 4 is satisfied → YES
- Otherwise → NO

OUTPUT FORMAT:
Provide your assessment in the following markdown format:

```markdown
# Climate Legislation Assessment: CP 1.a Framework Climate Law

## Individual Criterion Evaluation

### Criterion 1: Strategic Direction for Decarbonisation
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether the law includes clear Paris Agreement goals or long-term decarbonisation targets]

### Criterion 2: Enshrined in Law
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether this is legislative rather than executive]

### Criterion 3: Sets Out Obligations
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of which obligations are present, if any]

### Criterion 4: Exceptional Case
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether broad environmental law + executive strategy combination exists]

## Final Assessment

**Overall Result:** [YES/NO]

**Logic Applied:** [Explain whether criteria 1+2+3 are satisfied OR criterion 4 is satisfied]

**Conclusion:** [Brief summary of why the country does/does not have a framework climate law]
```"""),
    ("human", "Context: {context}\n\nPlease evaluate whether this country has a framework climate law based on the provided context.")
])

# Create the single-model chain
single_model_assessment_chain = comprehensive_assessment_prompt | large_context_llm


# Run the assessment
result = single_model_assessment_chain.invoke({"context": all_documents_context})
print(result.content)

# CP1b - Does the country’s framework climate law specify key accountability elements?


A country is assessed as ‘Yes’ if its framework climate law contains all three of the following accountability elements: 
1. Specification of who is accountable to whom for at least one stated obligation (e.g. accountability of executive to parliament, or private parties to executive authorities) 
2. Specification of how compliance is assessed for at least one stated obligation (e.g. transparency mechanisms in the form of monitoring, reporting and verification, parliamentary oversight, expert assessments, court proceedings) 
3. Specification of what happens in the case of non-compliance for at least one stated obligation (e.g. parliamentary intervention, judicial orders, financial penalties). 

For each of the accountability processes outlined these are examples of the types of relationships that usually qualify for the sub-questions. Each sub-question must be satisfied by at least one of the types of accountability.
1. Who is accountable to whom?
Examples of key relationships include:
a.	executive to parliament 
b.	executive to executive and/or administrative agencies 
c.	national to sub-national and sub-national to national
d.	executive to judiciary
e.	executive to expert bodies
f.	executive to citizens 
g.	private parties to citizens 
h.	private parties to executive authorities.
2. How is compliance assessed?
Some of the most common ways of assessing compliance are:
a.	transparency mechanisms in the form of monitoring, reporting and verification (MRV). Although this type of mechanism does not specify a decision-maker responsible for assessing whether an obligation has been met, this type of process is crucial for ensuring that there is political accountability for the implementation of climate law
b.	parliamentary oversight, which can also be used to assess the effectiveness of the legislation in achieving the stated aims
c.	Expert assessment
d.	Court proceedings
e.	Alternative Dispute Resolution.
3. What happens in the case of non-compliance?
It is relatively rare for countries to specify what happens in the case of non-compliance. Relevant practices identified in the laws can be grouped under certain categories. Some examples are:
a.	Parliamentary intervention
b.	Governmental or ministerial intervention
c.	Judicial orders
d.	Orders and fines by regulators
e.	Court imposed financial penalties
f.	Regulatory financial penalties
g.	Multi-stakeholder agreements.
Note that if a country has multiple laws assessed under the previous indicator, all are considered under this indicator on whether they together contain the three accountability elements above.

### 2.1 Retreive Context


### 2.2 Prompts


In [ ]:
# Define individual criterion evaluation prompts
cp1b_criterion_1_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a country's framework climate **Specification of who is accountable to whom for at least one stated obligation 
    (e.g. accountability of executive to parliament, or private parties to executive authorities)**
    
    These are examples of the types of relationships that usually qualify as having specification of who is accountable to whom:
        a.	executive to parliament 
        b.	executive to executive and/or administrative agencies 
        c.	national to sub-national and sub-national to national
        d.	executive to judiciary
        e.	executive to expert bodies
        f.	executive to citizens 
        g.	private parties to citizens 
        h.	private parties to executive authorities.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this country's framework climate law specify who is accountable to whom for at least one stated obligation?")
])

cp1b_criterion_2_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a country's framework climate law contains the following accountability element:
    **specification of how compliance is assessed for at least one stated obligation 
    (e.g. transparency mechanisms in the form of monitoring, reporting and verification, parliamentary oversight, expert assessments, court proceedings)**
        
    Some of the most common ways of assessing compliance are:
    a.	transparency mechanisms in the form of monitoring, reporting and verification (MRV). Although this type of mechanism does not specify a decision-maker responsible for assessing whether an obligation has been met, this type of process is crucial for ensuring that there is political accountability for the implementation of climate law
    b.	parliamentary oversight, which can also be used to assess the effectiveness of the legislation in achieving the stated aims
    c.	Expert assessment
    d.	Court proceedings
    e.	Alternative Dispute Resolution.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nIs this country's framework climate law specification of how compliance is assessed for at least one stated obligation?")
])

cp1b_criterion_3_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a country's framework climate law contains the following accountability element:
        **Specification of what happens in the case of non-compliance for at least one stated obligation (e.g. parliamentary intervention, judicial orders, financial penalties).**
        
    It is relatively rare for countries to specify what happens in the case of non-compliance. Relevant practices identified in the laws can be grouped under certain categories. Some examples are:
    a.	Parliamentary intervention
    b.	Governmental or ministerial intervention
    c.	Judicial orders
    d.	Orders and fines by regulators
    e.	Court imposed financial penalties
    f.	Regulatory financial penalties
    g.	Multi-stakeholder agreements.
    Note that if a country has multiple laws assessed under the previous indicator, all are considered under this indicator on whether they together contain the three accountability elements above.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this country's framework climate law specify what happens in the case of non-compliance for at least one stated obligation?")
])

# Final assessment prompt
cp1b_final_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' if the country's framework climate law specifies ALL THREE of the following key accountability elements:
        Criterion 1: Specification of who is accountable to whom for at least one stated obligation
        Criterion 2: Specification of how compliance is assessed for at least one stated obligation
        Criterion 3: Specification of what happens in the case of non-compliance for at least one stated obligation

    If YES to all criteria, provide a final 'YES' or 'NO' answer with reasoning."""),
    ("human", """Individual criterion assessments:
    Criterion 1 (who is accountable to whom): {criterion_1_result}
    Criterion 2 (compliance is assessed): {criterion_2_result}
    Criterion 3 (case of non-compliance): {criterion_3_result}
    
    What is the final assessment?""")
])

In [ ]:
# add implementation here if cp1a code works